### EXAMEN - Convocatoria 1 - Programación
Resolver el siguiente problema de regresión lineal.

#### 1) Generación de los datos (1 punto):

Mediante la función `make_regressión` de sklearn, consultar su como emplearla para generar un conjunto de 2000 datos, 10 características, un ruido de 1 y fijar el parámetro ramdom_state=42.

In [3]:
# Generación de datos
from sklearn.datasets import make_regression

# Generar datos de regresión
X, y = make_regression(n_samples=2000, n_features=10, noise=1, random_state=42)

# Verificar las dimensiones de X e y
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

Dimensiones de X: (2000, 10)
Dimensiones de y: (2000,)


#### 2) Partición de datos externa (1 punto)
Realizar una partición externa de tipo hold-out seleccionando un 20% de los datos para test (fijar una semilla en 42).

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las dimensiones de los conjuntos de entrenamiento y test
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

Dimensiones de X_train: (1600, 10)
Dimensiones de X_test: (400, 10)
Dimensiones de y_train: (1600,)
Dimensiones de y_test: (400,)


#### 3) Estandarización de los datos de train y test (1 punto)
Utilizar el método StandardScaler().

In [ ]:
from sklearn.preprocessing import StandardScaler

# Inicializar el StandardScaler
scaler = StandardScaler()

# Ajustar el scaler a los datos de entrenamiento y transformar los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la misma transformación a los datos de test
X_test_scaled = scaler.transform(X_test)

# Verificar las dimensiones de los conjuntos de entrenamiento y test
print("Dimensiones de X_train_scaled:", X_train_scaled.shape)
print("Dimensiones de X_test_scaled:", X_test_scaled.shape)

Dimensiones de X_train_scaled: (1600, 10)
Dimensiones de X_test_scaled: (400, 10)


#### 4) Creación de modelos e hiperparametrización mediante el método de sklearn GridSearchCV (4 puntos):

Instrucciones:

- Cree los siguientes modelos de regresión en un diccionario:
  - Regresión Lineal (OLS).
  - Regresión Lasso
  - Regresión Rígida
  - Vecinos más cercanos (KNN)

- Cree las siguiente métricas en un diccionario para evaluar:
  - MAE
  - MSE
  - RMSE
  - R2

- Haga una búsqueda de los siguientes hiperparámetros mediante GridSearchCV:

```
# Hiperparametros
parameters = {'OLS':{},
              'Lasso':{'alpha':(0.1,0.5,1,5,10,50,100)},
              'Ridge':{'alpha':(0.1,0.5,1,5,10,50,100)},
              'KNN':{'n_neighbors':np.arange(1,15),
                     'weights':('uniform','distance'),
                     'metric':('euclidean','manhattan','cosine')
                     }
              }
```

In [ ]:
from math import sqrt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np

# Crear modelos de regresión en un diccionario
models = {
    "OLS": LinearRegression(),
    "Lasso": Lasso(random_state=42),
    "Ridge": Ridge(random_state=42),
    "KNN": KNeighborsRegressor()
}

# Crear metricas de evaluación en un diccionario
metrics = {
    "MAE": mean_absolute_error,
    "MSE": mean_squared_error,
    "RMSE": lambda y_true, y_pred: sqrt(mean_squared_error(y_true, y_pred)),
    "R2": r2_score
}

# Hiperparámetros
parameters = {
    'OLS':{},
    'Lasso':{'alpha':(0.1,0.5,1,5,10,50,100)},
    'Ridge':{'alpha':(0.1,0.5,1,5,10,50,100)},
    'KNN':{'n_neighbors':np.arange(1,15), 'weights':('uniform','distance'), 'metric':('euclidean','manhattan','cosine')}
}

# Crear un diccionario para guardar los resultados de la evaluación de los modelos
results = {}

# Crear un diccionario para guardar los resultados de la búsqueda de hiperparámetros
best_params = {}

# Iterar sobre los modelos
for model_name, model in models.items():
    print(f"Modelo: {model_name}")
    results[model_name] = {}
    best_params[model_name] = {}

    # Iterar sobre las métricas
    for metric_name, metric in metrics.items():
        print(f"\tMétrica: {metric_name}")

        # Crear un objeto scorer a partir de la métrica
        if metric_name == "RMSE":
            # Para RMSE, usamos greater_is_better=False porque queremos minimizar el error
            scorer = make_scorer(metric, greater_is_better=False)
        else:
            scorer = make_scorer(metric)

        # Inicializar el GridSearchCV con el modelo, los hiperparámetros y la métrica a evaluar
        grid_search = GridSearchCV(model, parameters[model_name], scoring=scorer, cv=KFold(n_splits=5, shuffle=True, random_state=42))

        # Ajustar el GridSearchCV a los datos de entrenamiento
        grid_search.fit(X_train_scaled, y_train)
        
        # Guardar los mejores hiperparámetros
        best_params[model_name][metric_name] = grid_search.best_params_
        
        # Evaluar el modelo con los mejores hiperparámetros en los datos de test
        y_pred = grid_search.predict(X_test_scaled)
        results[model_name][metric_name] = metric(y_test, y_pred)

        print(f"\t\tResultado: {results[model_name][metric_name]:.4f}")
        print(f"\t\tMejores hiperparámetros: {best_params[model_name][metric_name]}")


Modelo: OLS
	Métrica: MAE
		Resultado: 0.7731
		Mejores hiperparámetros: {}
	Métrica: MSE
		Resultado: 0.9534
		Mejores hiperparámetros: {}
	Métrica: RMSE
		Resultado: 0.9764
		Mejores hiperparámetros: {}
	Métrica: R2
		Resultado: 1.0000
		Mejores hiperparámetros: {}
Modelo: Lasso
	Métrica: MAE
		Resultado: 150.7415
		Mejores hiperparámetros: {'alpha': 100}
	Métrica: MSE
		Resultado: 35093.2635
		Mejores hiperparámetros: {'alpha': 100}
	Métrica: RMSE
		Resultado: 1.0173
		Mejores hiperparámetros: {'alpha': 0.1}
	Métrica: R2
		Resultado: 1.0000
		Mejores hiperparámetros: {'alpha': 0.1}
Modelo: Ridge
	Métrica: MAE
		Resultado: 8.6452
		Mejores hiperparámetros: {'alpha': 100}
	Métrica: MSE
		Resultado: 115.6398
		Mejores hiperparámetros: {'alpha': 100}
	Métrica: RMSE
		Resultado: 0.9759
		Mejores hiperparámetros: {'alpha': 0.1}
	Métrica: R2
		Resultado: 1.0000
		Mejores hiperparámetros: {'alpha': 0.1}
Modelo: KNN
	Métrica: MAE
		Resultado: 98.6919
		Mejores hiperparámetros: {'metric': 'co

#### 5) Evaluación de los modelos sobre el conjunto de test (2 puntos)
- Entrenar los modelos anteriores utilizando todos los datos de entrenamiento y quédese con el mejor modelo de cada uno de los modelos de regresión arrojados por GridSearchCV. Es decir, debe tener un mejor modelo por cada regresión (OLS, Lasso, Rígida y KNN)
- Evaluar su rendimiento sobre el conjunto de test mostrando para cada uno de los modelos

In [ ]:
# Crear un diccionario para guardar los mejores modelos
from sklearn.model_selection import cross_val_score


best_models = {}

# Iterar sobre los modelos y entrenar con los mejores hiperparámetros
for model_name, model in models.items():
    # Crear un modelo con los mejores hiperparámetros 
    best_model = model.set_params(**best_params[model_name]['R2']) # Usamos R2 como métrica de referencia

    # Entrenar el modelo con todos los datos de entrenamiento
    best_model.fit(X_train_scaled, y_train)
    
    # Guardar el mejor modelo
    best_models[model_name] = best_model

# Evaluar los mejores modelos en el conjunto de test
for model_name, model in best_models.items():
    print(f"Modelo: {model_name}")
    
    # Predecir los valores de y en el conjunto de test
    y_pred = model.predict(X_test_scaled)
    
    # Iterar sobre las métricas
    for metric_name, metric in metrics.items():
        # Calcular la métrica
        result = metric(y_test, y_pred)
        print(f"\t{metric_name}: {result:.4f}")

print("\n--------------------------\n")
print("Cross-validation interno con 5 bolsas")

# Hacer cross-validation para cada modelo y calcular la accuracy para seleccionar el mejor modelo
for model_name, model in best_models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring=make_scorer(r2_score))
    # Calcular la accuracy 
    print(model_name + ' Accuracy: %0.4f +/- %0.4f' % (scores.mean(), scores.std()))



Modelo: OLS
	MAE: 0.7731
	MSE: 0.9534
	RMSE: 0.9764
	R2: 1.0000
Modelo: Lasso
	MAE: 0.8031
	MSE: 1.0349
	RMSE: 1.0173
	R2: 1.0000
Modelo: Ridge
	MAE: 0.7727
	MSE: 0.9524
	RMSE: 0.9759
	R2: 1.0000
Modelo: KNN
	MAE: 50.0401
	MSE: 4579.8916
	RMSE: 67.6749
	R2: 0.8720

--------------------------

Cross-validation interno con 5 bolsas
OLS Accuracy: 1.0000 +/- 0.0000
Lasso Accuracy: 1.0000 +/- 0.0000
Ridge Accuracy: 1.0000 +/- 0.0000
KNN Accuracy: 0.8622 +/- 0.0085


#### 6) Interpretación de resultados (1 puntos)
* Justifica brevemente cuál de los modelos utilizarías para ponerlo en producción

Los algoritmos de Regresión Lineal (OLS), Lasso y Ridge tienen un rendimiento muy similar, con valores de R2 practicamente igual a 1 y una gran accuracy en validación interna cruzada, lo cual indica que son modelos estables y que generalizan bien. KNN tiene un desempeño bastante peor por lo que queda descartado.
Entre estos tres algoritmos (OLS, Lasso y Ridge), vemos que Lasso tiene errores ligeramente más altos que OLS y Ridge con lo que también podríamos descartarlo en base a esto.
Como comparación final, vemos que OLS y Ridge son prácticamente identicos en cuanto a métricas, Ridge teniendo MAE, MSE y RMSE ligerísimamente inferiores a OLS (diferencias prácticamente negligibles)

Basado en este análisis, podemos concluir que ambos modelos son buenas elecciones para desplegar en Producción pero debido a la simplicidad de la Regresión Lineal (OLS), lo fácil de interpretar que resulta, su eficiencia computacional y que no necesita de ajuste de hiperparámetros, lo cuál simplifica su mantenimiento, concluímos que **OLS** debe ser el modelo elegido.